This notebooks shows how to pretrain any language model easily


1. Pretrain Roberta Model: this notebook
2. Finetune Roberta Model: https://www.kaggle.com/maunish/clrp-pytorch-roberta-finetune<br/>
   Finetune Roberta Model [TPU]: https://www.kaggle.com/maunish/clrp-pytorch-roberta-finetune-tpu
3. Inference Notebook: https://www.kaggle.com/maunish/clrp-pytorch-roberta-inference
4. Roberta + SVM: https://www.kaggle.com/maunish/clrp-roberta-svm


In [ ]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from transformers import (AutoModel,AutoModelForMaskedLM, 
                          AutoTokenizer, LineByLineTextDataset,
                          DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments)

In [ ]:
!pip install xlrd

In [ ]:
!pip install openpyxl

In [ ]:
df0 = pd.read_excel('../input/commonlit-dataset/0_replace.xlsx', engine='openpyxl')

In [ ]:
def text2(data):
    #data['external_text'] = data['external_text'].apply(lambda x: x.replace('\n',''))
    out = '\n'.join(data.tolist())
    return out

In [ ]:
import numpy as np

In [ ]:
df0 = df0.external_text.replace(np.nan, '', regex=True)

In [ ]:
df4 = pd.read_csv('../input/commonlit-dataset/4_children_stories.csv', encoding='latin1')

In [ ]:
#train_data = pd.read_csv('../input/commonlit-dataset/train.csv')
#test_data = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
df1 = pd.read_csv('../input/commonlit-dataset/1_train.csv')
df2 = pd.read_csv('../input/commonlit-dataset/2_test.csv')
df3 = pd.read_csv('../input/commonlit-dataset/3_children_books.csv')
df5 = pd.read_csv('../input/commonlit-dataset/5_enwiki.csv')
df6 = pd.read_csv('../input/commonlit-dataset/6_simwiki.csv')
df7 = pd.read_csv('../input/commonlit-dataset/7_weebit.csv')

#data = pd.concat([train_data,test_data])

In [ ]:
def text(data):
    data['excerpt'] = data['excerpt'].apply(lambda x: x.replace('\n',''))
    out = '\n'.join(data.excerpt.tolist())
    return out

In [ ]:
txt0 = text2(df0)
txt1 = text(df1)
txt2 = text(df2)
txt3 = text(df3)
txt4 = text(df4)
txt5 = text(df5)
txt6 = text(df6)
txt7 = text(df7)
# txt8 = text(df8)

In [ ]:
txt = txt0 + txt1 + txt2 + txt3 + txt4 + txt5 + txt6 + txt7

In [ ]:
#data.head()

In [ ]:
#data['url_legal'] = data['url_legal'].fillna('')
#data['license'] = data['license'].fillna('')

In [ ]:
#data['excerpt'] = data['excerpt'].apply(lambda x: x.replace('\n','')) # replace new line with nothing

In [ ]:
#data['all'] = data['url_legal'] + ', ' + data['license'] + ', '+ data['excerpt']

In [ ]:
#data['all2'] = data['all'].values

In [ ]:
#text  = '\n'.join(data.all2.tolist()) # add \n in between each 

In [ ]:
with open('text.txt','w') as f:
    f.write(txt)

In [ ]:
model_name = 'roberta-base'
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained('./clrp_roberta_base');

In [ ]:
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="text.txt", #mention train text file here
    block_size=256)

valid_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="text.txt", #mention valid text file here
    block_size=256)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

training_args = TrainingArguments(
    output_dir="./clrp_roberta_base_chk", #select model path for checkpoint
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy= 'steps',
    save_total_limit=2,
    eval_steps=200,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end =True,
    prediction_loss_only=True,
    report_to = "none")

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset)

In [ ]:
trainer.train()
trainer.save_model(f'./clrp_roberta_base')